# Introduction

## Geospatial data I/O

Data analysis tasks involve reading data stored in files on disks, servers in the cloud, or recorded by sensors. Also, we need to save the results of our analysis or datasets we have generated to files. 

### Task

In this lab we will write a program that can convert crop yield data collected from a harvester into a geospatial data structure for vector data and write this data to disk using a geospatial file format. You will also write a program that can read geospatial image (raster) data from files into array objects that can be analysed in Python programs. 

This lab will introduce:

* geospatial data file formats
* techniques for reading and writing from geospatial data from and to files
* Python data structures for representing vector data (`GeoDataFrame` objects)
* Python data structures for representing raster data (NumPy `ndarray` objects)


## Setup

### Run the labs

You can run the labs locally on your machine or you can use cloud environments provided by Google Colab or Binderhub. **If you're working with Google Colab be aware that your sessions are temporary and you'll need to take care to save, backup, and download your work.**

<a href="https://colab.research.google.com/github/data-analysis-3300-3003/colab/blob/main/lab-2-self-guided.ipynb" target="_blank">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

### Download data

If you need to download the date for this lab, run the following code snippet. 

In [ ]:
import os

if "week-2" not in os.listdir(os.getcwd()):
    os.system('wget "https://github.com/data-analysis-3300-3003/data/raw/main/data/week-2.zip"')
    os.system('unzip "week-2.zip"')

### Working in Colab

If you're working in Google Colab, you'll need to install the required packages that don't come with the colab environment.

In [ ]:
if 'google.colab' in str(get_ipython()):
    !pip install geopandas
    !pip install pyarrow
    !pip install mapclassify
    !pip install rasterio

### Import modules

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import rasterio
import json
import os
import folium
import plotly.express as px
import plotly.io as pio
import pprint

# setup renderer
if 'google.colab' in str(get_ipython()):
    pio.renderers.default = "colab"
else:
    pio.renderers.default = "jupyterlab"

## Geospatial data

Geospatial data is used to take geographic features or phenomenon and represent them as data in computer program or file. 

There are two main types of geospatial data: 

* **vector data** - point, line, or polygon geometries
* **raster data** - images and arrays

There are two components to geospatial data:

* **Positional information** describing location, shape, and extent (e.g. an `(x, y)` coordinate pair representing the location of a weather station).
* **Attribute information** describing characteristics of the phenomenon or entity (e.g. a name:value pair recording the name of the weather station `name:'Perth Airport'`).

#### Raster data model

Raster data breaks the Earth's surface up into a grid of cells (pixels). Each pixel is assigned a value that corresponds to the geographic feature or phenomenon of interest. For example, pixels in a raster precipitation dataset would be assigned a numeric value that represents the amount of precipitation that fell at that location. Pixels in a land cover map would have an integer value that corresponds to a particular land cover class label. The values assigned to pixels in a raster dataset are the attribute information. 

![](https://github.com/data-analysis-3300-3003/figs/raw/main/week-2-raster-data-lc.png)

The size of the pixels relative to their position on the Earth's surface determines the spatial detail that can be resolved in the raster dataset. A land cover map with pixels that represent a 1 km x 1 km portion of the Earth's surface will not be able to identify features such as individual buildings.

<!-- The figure below shows the 2018 European Space Agency (ESA) Climate Change Initiative (CCI) land cover map. Each pixel represents a 300 m x 300 m area on the Earth’s land surface and a pixel can only represent a single land cover type. If you look at the bottom two zoomed in maps you can see some limitations of representing land cover using 300 m x 300 m spatial resolution raster data. The shape of land cover features are poorly represented by the “block-like” arrangement of pixels and there is variation in land cover within a single pixel (a mixed pixel problem).

![](https://github.com/data-analysis-3300-3003/figs/raw/main/week-2-raster-data-model.png) -->

#### Vector data model

Vector data uses point, line, or polygon geometries to represent geographic features. 

**Coordinate pairs:** point locations or the vertices in lines and polygons are represented using coordinate pairs. The coordinate pairs indicate where that feature is located on the Earth's surface (relative to an origin); longitude and latitute are commonly used as coordinate pairs. 

**Attribute information:** vector data also stores non-spatial attribute information which describe characteristics of the geographic phenomenon or entity represented by the geometry feature.

![](https://github.com/data-analysis-3300-3003/figs/raw/main/week-2-vector-data.jpg)

## GeoPandas GeoDataFrame

A GeoPandas `GeoDataFrame` is a tabular data structure for storing vector geospatial data and is based on a regular pandas `DataFrame`. 

A `GeoDataFrame` consists of columns of non-spatial attributes similar to a pandas `DataFrame`. However, a `GeoDataFrame` also has a `geometry` column which is a `GeoSeries` of geometries for the spatial data associated with each row.

In Python, geometries are represented as <a href="https://shapely.readthedocs.io/en/stable/geometry.html" target="_blank">Shapely</a> `Geometry` objects. The `geometry` column in a GeoPandas `GeoDataFrame` is a `Series` of Shapely `Geometry` objects. Printing a <a href="https://shapely.readthedocs.io/en/stable/geometry.html" target="_blank">Shapely</a> `Geometry` object returns a Well Known Text (WKT) string description of the geometry (e.g. `POINT (0, 1)`). The `geometry` column of a `GeoDataFrame` (or a `GeoSeries`) can be viewed as a sequence of Shapely `Geometry` objects:

```
a_geoseries = [POINT (0, 1), POINT (0, 2), POINT (2, 3)]
```

Shapely provides tools for representing geometries in Python programs. It does not provide tools for reading geometry data from disk or handling attribute data. GeoPandas `GeoDataFrame` and `GeoSeries` combine Shapely's functionality for handling geometries with tools for reading and writing vector data, handling attributes, and visualisation. Therefore, we will focus on using `GeoDataFrame`s in these labs.  

Let's convert a CSV file with longitude, latitude, and elevation columns into a `GeoDataFrame`. First, let's read the CSV file in as a pandas `DataFrame`. 

In [ ]:
elev_df = pd.read_csv(os.path.join(os.getcwd(), "week-2", "week-2-bf66-elevation.csv"))
elev_df.head()

Now, let's use the longtitude and latitude columns in the `DataFrame` to convert the elevation data into a GeoPandas `GeoDataFrame`.

In [ ]:
# Convert the elevation data to a spatial format
points = gpd.points_from_xy(elev_df["Lon"], elev_df["Lat"], crs="EPSG:4326")
print(f"points is of type {type(points)}")

elev_gdf = gpd.GeoDataFrame(elev_df, geometry=points)
print(f"elev_gdf is of type {type(elev_gdf)}")

elev_gdf.head()

#### Recap quiz

You will need to refer to the GeoPandas documentation to answer these questions.

<details>
    <summary><b>What does executing the <code>GeoDataFrame</code> method <a href="https://geopandas.org/en/stable/docs/reference/api/geopandas.points_from_xy.html" target="_blank"><code>points_from_xy()</code></a> return?</b></summary>
<code>points_from_xy()</code> expects a pandas <code>Series</code> objects for x and y coordinates and coordinate reference system. It will return to a GeoPandas <code>GeometryArray</code> object which stores a POINT geometry object for each x and y pair and can be converted into a <code>GeoSeries</code> object.
</details>

<p></p>
    
<details>
    <summary><b>The <a href="https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.html#geopandas.GeoDataFrame" target="_blank"><code>GeoDataFrame()</code></a> constructor function can take three arguments. What are these arguments and how do they enable the creation of a <code>GeoDataFrame</code> object?</b></summary>
    The <code>GeoDataFrame()</code> constructor function requires a pandas <code>DataFrame</code> as its first argument. This data is the non-spatial attributes. The second (optional) argument is a GeoPandas  object which stores <code>geometry</code> objects associated with each row (this could also be a string denoting the column of a <code>DataFrame</code> storing geometries. The third (optional) argument is a crs denoting the coordinate reference system for the geometry data.
</details>

## GeoJSON

So far we have been reading and writing non-spatial tabular data. Parquet and CSV files are well suited for storing tabular data. 

However, there are different ways of structuring data, some datasets don't lend themselves to tabular structures, and there are different uses of data files than storage on disk. 

JSON data (JavaScript Object Notation for its full name) is a widely used format for data interchange (exchanging data between programs, computers, clients, and servers). 

JSON represents data as key:value pairs enclosed within curly brackets `{}` (you might notice the similarity with Python's dictionary data structure). 

This is an example of JSON data:

```
{
    "title": "Introducing JSON",
    "url": "https://www.json.org/json-en.html"
}
```

The values in JSON data can include text (strings), numbers, arrays (lists), and nested JSON objects. Like the CSV format, JSON is a text based format where human readable characters are encoded in binary using UTF-8 or UTF-16.

GeoJSON is an extension of the JSON format for storing and exchanging spatial data. One of GeoJSON's uses is sending spatial data to web browsers to render as layers on web maps.

GeoJSON represents geographic features as vector data (points, lines, and polygon geometries) and can also store non-spatial attribute information.
 
Spatial data in GeoJSON are represented using `geometry` types which include:

`Point`

```
{"type": "Point", "coordinates": [1, 1]}
```

`LineString`

```
{"type": "LineString", "coordinates": [[1, 1], [2, 2]]}
```

`Polygon`

```
{"type": "Polygon", "coordinates": [[[1, 1], [2, 2], [1, 2], [1, 1]]]}
```

`Feature` types include attribute data as `properties` with `geometry` types.

```
{
    "type": "Feature",
    "geometry": {
        "type": "Point",
        "coordinates": [0, 0]
    }, 
    "properties": {
        "name": "Perth Airport"
    }
}
```

A `FeatureCollection` is a collection of `Feature`s.

```
{
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": [0, 0]
            }, 
            "properties": {
                "name": "Perth Airport"
            }
        },
        {
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": [10, 1]
            }, 
            "properties": {
                "name": "Broome Airport"
            }
        }
        
    ]
}
```

You can read <a href="https://macwright.com/2015/03/23/geojson-second-bite.html#featurecollection" target="_blank">More than you ever wanted to know about GeoJSON</a> for a description of the GeoJSON format. 

Let's get the first two rows of the `GeoDataFrame` and convert them to GeoJSON format. `GeoDataFrame`s have a `to_json()` method which can be used to convert the data in the `GeoDataFrame` into a string object in GeoJSON format. 

In [ ]:
# Get the first two rows of the elevation GeoDataFrame and convert to GeoJSON
elev_gdf_2 = elev_gdf.iloc[0:2, :]
elev_gdf_2

In [ ]:
elev_geojson_2 = elev_gdf_2.to_json()
print(f"The GeoJSON data is stored as a {type(elev_geojson_2)} type object")
print("")
pprint.pprint(elev_geojson_2)

In Python, the GeoJSON data that we have generated from our `GeoDataFrame` is stored as a string object. GeoJSON (and JSON) is a text-based data similar to CSV files. However, unlike the CSV format where data has a tabular structure with records arranged by row the GeoJSON data is based around nested objects of key:value pairs.

As we have subsetted the first two rows of our `GeoDataFrame` and converted them to GeoJSON we have generated a `FeatureCollection` object with two `Feature`s. 

Each row in the `GeoDataFrame` is converted to a `Feature` and each `Feature` has the column values per row stored in a `properties` object - these are the non-spatial attributes associated with each `Point` feature. The spatial information is stored in a `geometry` object which contains two key:value pairs. The value associated with the `type` key tells us this is a `Point` geometry and the array value associated with `coordinates` key defines the location.

Compare the tabular display of the `GeoDataFrame` to the print of the GeoJSON to see how the non-spatial and spatial information in the table structure is converted to the GeoJSON nested format. 

We can save a `GeoDataFrame` to GeoJSON using the `GeoDataFrame`'s `to_file()` method and setting the driver argument to GeoJSON.

In [ ]:
# Save the elevation GeoDataFrame to a GeoJSON file
elev_gdf.to_file(os.path.join(os.getcwd(), "week-2", "week-2-bf66-elevation.geojson"), driver="GeoJSON")

Check the GeoJSON file has saved to the directory specified. As it is text data, if you click on it you should be able to inspect its format in a text editor. 

#### Recap quiz

<details>
    <summary><b>Identify two differences between the GeoJSON file format and a GeoPandas <code>GeoDataFrame</code></b></summary>
<ul>
<li>A <code>GeoDataFrame</code> is used to store geospatial data in memory for Python programs. A GeoJSON file format describes how geospatial data should be encoded when it is stored on disk.</li>
<li>A <code>GeoDataFrame</code> uses a tabular structure to organise non-spatial and spatial attributes with each row corresponding to a feature. GeoJSON format uses dictionary-like structure of key:value pairs with geographic data (coordinates) stored as values with a <code>geometry</code> key and attribute data stored as values with a <code>properties</code> key.</li>
</ul>
</details>


**You saved the elevation data to a GeoJSON file at this path: `os.path.join(os.getcwd(), "week-2", "week-2-bf66-elevation.geojson")`.**

**Head to the <a href="https://geopandas.org/en/stable/getting_started/introduction.html#Reading-and-writing-files" target="_blank">GeoPandas documentation</a> and look up how to read files into `GeoDataFrame` objects. Read the *elevation.geojson* file into a `GeoDataFrame` referenced by the variable `elev_from_file`.**

In [ ]:
## ADD CODE HERE ##

<details>
    <summary><b>answer</b></summary>

```{python}
elev_from_file = gpd.read_file(os.path.join(os.getcwd(), "week-2", "week-2-bf66-elevation.geojson"))
elev_from_file.head()
```
    
Note, this answer assumes GeoPandas has been imported as gpd. 
</details>

**Write the data referenced by `elev_from_file` to disk as a GeoPackage.**

In [ ]:
## ADD CODE HERE ##

<details>
    <summary><b>answer</b></summary>
    
```{python}
elev_from_file.to_file(os.path.join(os.getcwd(), "week-2", "week-2-bf66-elevation.gpkg"), driver="GPKG")
```
</details>

## GeoTIFF

We have demonstrated how we can read and write tabular data and vector geospatial data from and to files. However, many geospatial datasets are based on the raster data model where values are assigned to pixels and pixels represent locations on the Earth's surface. 

A common source of raster data are remote sensing images captured by sensors on uncrewed aerial vehicles, aircraft, or satellites. Optical remote sensing images store the measured reflectance of light off the Earth's land surface in different wavelenghts. Raster remote sensing images are often stored using the <a href="https://gdal.org/drivers/raster/gtiff.html" target="_blank">GeoTIFF</a> format. 

A GeoTIFF file is based on the Tagged Image File Format (or .tiff file) which is a general format for storing image data. A TIFF file comprises:

* a **TIFF header** which includes 8 bytes that tell us that the file is in TIFF format and where in the file (what byte number / byte offset from 0) the first Image File Directory is stored.
* **Image File Directories** which contains image metadata, a pointer to where the image data is in the file (what byte number / byte offset from 0), and the location of the next Image File Directory if there is more than one image stored in the TIFF file. Metadata is stored as fields which comprise a TIFF tag and it's corresponding value.
* **Image Data** - the values associated with each pixel in the image. A single TIFF file can store multiple images.

![](https://github.com/data-analysis-3300-3003/figs/raw/main/week-2-geotiff.jpg)

GeoTIFF files include extra information (metadata) as tags which describe the coordinate reference system (CRS) of the image data (i.e. where on the Earth's surface the image data corresponds to), spatial resolution, no data values, and various other configurations described <a href="https://gdal.org/drivers/raster/gtiff.html" target="_blank">here</a>. 

GeoTIFF files can store multiple images (i.e. raster layers) in a single file. This makes them well suited for storing remote sensing image data where each raster layer corresponds to measured reflectance in a particular wavelength. 

We can use the functions provided by the <a href="https://rasterio.readthedocs.io/en/latest/" target="_blank">rasterio</a> package to read and write raster data in Python.

We can use rasterio to create a file connection to raster data stored in a GeoTIFF file and use this file connection object to read the raster data into a Python data structure. Rasterio reads raster data from GeoTIFF files on disk into NumPy `ndarray`s in memory. 

The GeoTIFF file *week-2-s2-summer-2020.tif* stores remote sensing data covering the same field that the elevation and canola yield data were collected from. The remote sensing data was captured by the <a href="https://sentinel.esa.int/web/sentinel/missions/sentinel-2" target="_blank">European Space Agency's Sentinel-2 satellite</a> (10 m spatial resolution). 

We use the `with` statement and context managers to `open()` connections to GeoTIFF files to `read()` raster data from them using rasterio. As we're using a context manager to handle connections to the file we don't need to call `close()` on the `file` connection object.

In [ ]:
# path the the GeoTIFF file
s2_path = os.path.join(os.getcwd(), "week-2", "week-2-s2-summer-2020.tif")

# open the GeoTIFF file and read its metadata and image data
with rasterio.open(s2_path) as src:
    meta = src.meta
    rgb = src.read([4, 3, 2])
    red_band = src.read(4)
    green_band = src.read(3)
    blue_band = src.read(2)

The `file` object `src` is the connection to the GeoTIFF file and it has a `meta` property. The `meta` property stores metadata that describes the raster data in the GeoTIFF file. Let's explore this metadata.

In [ ]:
pprint.pprint(meta)

From the `meta` object we can see the data type of the raster:

In [ ]:
print(meta["dtype"])

We can also see the dimensions of each raster band:

In [ ]:
print(f"width: {meta['width']}") 
print(f"height: {meta['height']}") 

The `count` property of the `meta` object is the number of bands or raster layers in the GeoTIFF file:

In [ ]:
print(f"number of bands: {meta['count']}")

And, we can see the coordinate reference system of the raster data:

In [ ]:
print(meta["crs"])

### NumPy `ndarray`s

Calling the `read()` method of the `file` connection object to the GeoTIFF file reads the raster data into a NumPy `ndarray` object. We can specify a number as an argument to the `read()` method to indicate which raster bands we want to read from the GeoTIFF file. For example, `read(2)` will read the second band. If we specify a list of numbers we will read several bands into a multidimensional `ndarray` object. 

NumPy is a library used for scientific and numerical computing and is based around an N-dimensional `ndarray` object. An `ndarray` is a grid of elements of the same data type. The dimensions of a NumPy `ndarray` are called axes. NumPy `array`s can be created from sequences of values (e.g. stored in lists, tuples, other `ndarray`s).

We can create a simple 1-dimensional `ndarray` using the `array()` function.

In [ ]:
# create a 1D ndarray
arr1d = np.array([1, 2, 3])
arr1d

The rank (or number of dimensions) of a `ndarray` is the number of axes.

In [ ]:
# the rank (ndim) of an ndarry is the number of axes 
print(f"the rank of the ndarray is {arr1d.ndim}")

The `shape` of an `ndarray` tells us the size of each axis (how many elements are arranged along that axis).

In [ ]:
# the shape of the ndarray 
print(f"the shape of the ndarray is {arr1d.shape}")

A `ndarray` with 2-dimensions is a matrix with rows arranged on the 0 axis and columns arranged on the 1 axis. 

In [ ]:
# create a 2D ndarray
arr2d = np.array([[1, 2, 3], [4, 5, 6]])
arr2d

The `ndarray` pointed to by `arr2d` is a 2 x 3 matrix of numeric values. We can check this. 

In [ ]:
print(f"the rank of the ndarray is {arr2d.ndim}")
print(f"the shape of the ndarray is {arr2d.shape}")

`ndarray`s can be multidimensional. They can have more than two dimensions. Remote sensing images typically comprise multiple 2-dimensional arrays with each array corresponding to a raster of reflectance measured in a particular wavelength. This 3-dimensional raster data structure can be represented as a NumPy `ndarray` with the bands dimension on axis 0 (each band is a raster for a given wavelength), rows (height of each raster) on axis 1, and columns (width of each raster) on axis 2. 

Let's create a `ndarray` with 3-dimensions. 

In [ ]:
# create a 3D ndarray
arr3d = np.array([[[1, 2, 3], [4, 5, 6]], [[7, 8, 9], [10, 11, 12]]])
arr3d

In [ ]:
print(f"the rank of the ndarray is {arr3d.ndim}")
print(f"the shape of the ndarray is {arr3d.shape}")

The concept of N-dimensional arrays can be extended further. For example, a 4-dimensional `ndarray` could store a sequence of 3-dimensional `ndarray`s where the fourth dimension is time and the object represents remote sensing images captured across multiple dates. 

#### Recap quiz

Execute the following code to read in a normalised difference vegetation index (NDVI) layer computed for the same field from a GeoTIFF file and use this data to answer the following questions. 

In [ ]:
# path to the ndvi file
s2_ndvi_path = os.path.join(os.getcwd(), "week-2", "week-2-ndvi-summer-2020.tif")

# open the GeoTIFF file and read its metadata and image data
with rasterio.open(s2_ndvi_path) as src:
    meta = src.meta
    ndvi = src.read(1)

<details>
    <summary><b>What is the name of the variable that points to the metadata for the NDVI data read from the GeoTIFF file?</b></summary>
<code>meta</code>
</details>

**Find out what type of object the data referenced by `meta` is?**

In [ ]:
## add code here ##

<details>
    <summary><b>answer</b></summary>
<code>print(type(meta))</code>
</details>

**Find out what type of object the data referenced by `ndvi` is?**

In [ ]:
## add code here ##

<details>
    <summary><b>answer</b></summary>
<code>print(type(ndvi))</code>
</details>

**Use the `meta` object to identify how many bands the raster data stored in the GeoTIFF *week-2-ndvi-summer-2020.tif* has?**

In [ ]:
## add code here ##

<details>
    <summary><b>answer</b></summary>
<code>print(f"There are {meta['count']} bands in the GeoTIFF file")</code>
</details>

**Find out how many raster bands have been read into the `ndarray` object referenced by `ndvi`?**

In [ ]:
## add code here ##

<details>
    <summary><b>answer</b></summary>
<p>One band has been read into the <code>ndarray</code> object pointed to by <code>ndvi</code>. This is a rank 2 `ndarray` with row and columns axes and no bands axis.</p>

```{python}
print(f"the rank of the ndvi ndarray is {ndvi.ndim}")
print(f"the shape of the ndvi ndarray is {ndvi.shape}")
```
</details>

<details>
    <summary><b>Do <code>meta["count"]</code> and the length of the bands axis of the <code>ndarray</code> object generated by a call to <code>read()</code> have to return the same value?</b></summary>
Not always. In this case they should return the same value as we are reading the one and only band stored in <em>week-2-ndvi-summer-2020.tif</em> in an <code>ndarray</code> object. However, the value at <code>meta["count"]</code> will tell us the number of bands in the file while the number of bands stored in the <code>ndarray</code> resulting from a call to <code>read()</code> is determined by the number passed into <code>read()</code>. 
</details>

### Visualising raster data

We can use the `imshow()` function from the Plotly Express package to render data in NumPy `ndarray`s as an image. The `imshow()` function expects a NumPy `ndarray` of raster values to display and a colour map (`color_continuous_scale`) that relates raster pixel values to a colour on the computer screen. 

In [ ]:
# Plot the red band
px.imshow(red_band, color_continuous_scale="Reds")

In [ ]:
# Plot the green band
px.imshow(green_band, color_continuous_scale="Greens")

In [ ]:
# Plot the Blue band
px.imshow(blue_band, color_continuous_scale="Blues")

#### Recap quiz

**Earlier we read NDVI data for this field into a `ndarray` object referenced by `ndvi`. Visualise this data using the `imshow()` function and select a suitable colour scale from the Plotly Expess built-in <a href="https://plotly.com/python/builtin-colorscales/" target="_blank">options</a>. NDVI values range from -1 to 1 with higher values associated with greener vegetation.**

In [ ]:
## add code here ##

<details>
    <summary><b>answer</b></summary>
<code>px.imshow(ndvi, color_continuous_scale="viridis")</code>
</details>

#### Colour

A particular colour is defined by the intensity of light in different parts of the visible spectrum (e.g. yellow is a mixture of light in red and green wavelengths).

Colour is represented by combinations (addition) of red, green, and blue light. Red, green, and blue are primary colours and combine to form white. An absence of red, green, and blue is black. Secondary colours can be formed by the addition of primary colours of varying intensities (e.g. yellow is the addition of red and green, magenta is the addition of red and blue, and cyan is the addition of green and blue). 

Computer displays consist of red, green, and blue sub-pixels, which when activated with different intensities, are perceived as different colours. The range of colours that can be displayed on a computer display is called the gamut. Colour in computer programs is represented as a three byte hexadecimal number with byte 1 corresponding to red, byte 2 corresponding to green, and byte 3 corresponding to blue. Each byte can take the range of 0 to 255 in decimal. 0 indicates the absence of a colour and 255 indicates saturation of that colour:

* white: 255 255 255
* black: 0 0 0
* red: 255 0 0 
* green: 0 255 00
* blue: 0 0 255

![Additive and subtractive colour models (source: CRCSI (2017))](https://github.com/data-analysis-3300-3003/figs/raw/main/week-2-colour-models-crcsi.png)

Computer displays represent colour through varying the intensity of sub-pixel displays of red, green, and blue light. Variability in data values in multiband rasters can be visualised by relating data values in one band to the intensity of one of the primary colours on the computer display. Visualising a multiband raster in this way creates an additive RGB or colour composite image - it is called a composite image because each pixel is a composite of red, green, and blue light.

Above we rendered the red, green, and blue band reflectance from the Sentinel-2 image separately. However, if we combine these reflectance measures into a composite image (e.g. where red reflectance is represented by sub-pixel intensity of red light) we can create a true colour image as if we were looking down on the Earth's surface with our eyes.

The `imshow()` function can take a multiband NumPy `ndarray` and returns an RGB image.

Above, we `read()` bands 4, 3, and 2 from the GeoTIFF file into NumPy `ndarray` referenced by `rgb`. Band 4 corresponds to red reflectance in Sentinel-2 images, band 3 corresponds to green reflectance, and band 2 corresponds to blue reflectance. We can pass `rgb` into the `imshow()` function to generate an RGB image of the field.

We can check that `rgb` has three bands by printing its `shape` property.

In [ ]:
print(f"The shape of rgb is {rgb.shape}")

If we check the `imshow()` <a href="https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.imshow.html" target="_blank">docs</a> we can see that it expects a NumPy `ndarray` where the bands (or channels as they are sometimes called with images) is the final axis. NumPy has a `moveaxis()` function where can move the bands dimension to the last axis position before plotting.

In [ ]:
# Plot the RGB image
px.imshow(np.moveaxis(rgb, 0, 2), contrast_rescaling="minmax")

#### Recap quiz

**At the file path noted below is a GeoTIFF file storing a normalised difference water index (NDWI) layer. Write a short program to read this raster data, identify the dimensions of the raster data, and visualise the data**

File path to NDWI data: `os.path.join(os.getcwd(), "week-2", "week-2-ndwi-summer-2020.tif")`

In [ ]:
## add code here ##

<details>
    <summary><b>answer</b></summary>

```{python}
s2_ndwi_path = os.path.join(os.getcwd(), "week-2", "week-2-ndwi-summer-2020.tif")

with rasterio.open(s2_ndwi_path) as src:
    meta = src.meta
    ndwi = src.read(1)

print(f"the width of the raster layer is {meta['width']}")
print(f"the height of the raster layer is {meta['height']}")
print(f"the number of bands in the raster layer is {meta['count']}")

px.imshow(ndwi, color_continuous_scale="YlGnBl")
```

</details>
